In [11]:
import dspy
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from tqdm.auto import tqdm

student_lm_name = "openai/gpt-5.1-2025-11-13"
teacher_lm_name = "openai/gpt-5.1-2025-11-13"

student_lm = dspy.LM(student_lm_name)
teacher_lm = dspy.LM(teacher_lm_name)

dspy.configure(lm=student_lm)

# Prepare DSPY Dataset

In [12]:
# Train val test split
lang = "eng"
# Load the training and validation data for subtask 1
train_df = pd.read_csv(f'./dev_phase/subtask1/train/{lang}.csv')
test_df = pd.read_csv(f'./dev_phase/subtask1/dev/{lang}.csv')
# Split train into train and validation sets
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [13]:
# Map the polarization numeric label to a string
POLARIZATION_MAP = {1: "polarization", 0: "no polarization"}

def make_dspy_examples(df, include_label: bool = True):
    examples = []
    for _, row in df.iterrows():
        kwargs = dict(
            sentence=row["text"],
        )
        if include_label and "polarization" in row:
            kwargs["polarization"] = POLARIZATION_MAP[row["polarization"]]
        example = dspy.Example(**kwargs).with_inputs("sentence")
        examples.append(example)
    return examples

# Create DSPY datasets
raw_train = make_dspy_examples(train_df, include_label=True)
raw_val = make_dspy_examples(val_df, include_label=True)
raw_test = make_dspy_examples(test_df, include_label=False)


# # For now take only 10% of each 
raw_train = raw_train[:int(len(raw_train) * 0.2)]
raw_val = raw_val[:int(len(raw_val) * 0.3)]
raw_test = raw_test[:int(len(raw_test) * 1)]

# Define Signature (Subtask 1)

In [14]:
from typing import Literal

class Polarization(dspy.Signature):
    """
    Polarization denotes stereotyping, vilification, dehumanization, deindividuation, or intolerance of other people’s views, beliefs, and identities. In this study, speeches and articles that are shared on social media that incite division, groupism, hatred, conflict, and intolerance are classified as containing polarization.
    Given this sentence, classify it as containing polarization or not."""

    sentence: str = dspy.InputField()
    polarization: Literal["polarization", "no polarization"] = dspy.OutputField(
        desc='Return "polarization" or "no polarization".',
        choices=["polarization", "no polarization"],
    )
classify = dspy.Predict(Polarization)

# Evaluation

In [15]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

POLARIZATION_MAP = {1: "polarization", 0: "no polarization"}

def label2id(label: str) -> int:
    return {v: k for k, v in POLARIZATION_MAP.items()}[label]

def id2label(i: int) -> str:
    return POLARIZATION_MAP[i]

def accuracy_metric(example, pred, trace=None):
    gold = example.polarization          # from your Examples
    guess = pred.polarization         # from Signature output
    return int(gold == guess)         # 1 = correct, 0 = incorrect

In [10]:
evaluate = dspy.Evaluate(
    devset=raw_val,
    metric=accuracy_metric,
    display_progress=True,
    display_table=True,   # nice overview
)
eval_result = evaluate(classify)
print("DSPy average accuracy metric:", eval_result.score)  # percentage

Average Metric: 62.00 / 73 (84.9%):  36%|███▌      | 73/202 [05:01<07:37,  3.55s/it] 

2025/12/04 17:37:29 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=None. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently None)  if the reason for truncation is repetition.


Average Metric: 165.00 / 202 (81.7%): 100%|██████████| 202/202 [12:46<00:00,  3.79s/it]

2025/12/04 17:45:07 INFO dspy.evaluate.evaluate: Average Metric: 165 / 202 (81.7%)


,sentence,example_polarization,pred_polarization,accuracy_metric
0,لطيفة فنانة كبيرة ومحترمة وهي مدرسة متجددة وراقية وجديرة التعليم,no polarization,no polarization,✔️ [1]
1,بعد بيتي يا الكدعان ، اوف اوف لو مكسرينها ، ما قصرتو اسباع \nالعراقي,polarization,polarization,✔️ [1]
2,ما افتهمت الكويت تقول كاظم الساهر انطلق من الكويت واحد يقول انطلق ...,no polarization,no polarization,✔️ [1]
3,من البيضة مهيوب الحر … \nمن لعب لاندية عالمية مثل برايتون و مالمو...,polarization,None,✔️ [0]
4,الاحتيال المالي يتفاقم.. هل تستطيع البنوك الصمود بمفردها؟,no polarization,no polarization,✔️ [1]
...,...,...,...,...
197,علم الطاقة \nالابراج \nالتاروت\nالاستبصار\nحرااااااااااام وشرك ولا...,polarization,polarization,✔️ [1]
198,فيديو .. وليد الفراج يعلق على خلع حسن معاذ للشورت شاهد على :,no polarization,no polarization,✔️ [1]
199,الجيش الإسرائيلي يصدر بيانا حول جثث الرهائن الأربعة,no polarization,no polarization,✔️ [1]
200,لطيفة اكبر منافقة تغني لمصر وتتودد للمصريين ايه النفاق ده يا لطيفة,polarization,polarization,✔️ [1]


DSPy average accuracy metric: 81.68


In [16]:
def eval_metrics_on_dataset(program, dataset):
    y_true, y_pred = [], []

    for ex in dataset:
        gold = ex.polarization
        pred = program(sentence=ex.sentence).polarization
        if pred is None:
            pred = "no polarization"
        y_true.append(label2id(gold))
        y_pred.append(label2id(pred))

    return {
        'f1_macro': f1_score(y_true, y_pred, average='macro'),
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='binary'),
        'recall': recall_score(y_true, y_pred, average='binary'),
        'f1_binary': f1_score(y_true, y_pred, average='binary'),
        'f1_micro': f1_score(y_true, y_pred, average='micro'),
    }

# metrics_val = eval_metrics_on_dataset(classify, raw_val)
# print("Validation metrics:", metrics_val)

# metrics_test = eval_metrics_on_dataset(classify, raw_test)
# print("Test metrics:", metrics_test)

In [17]:
ids = test_df["id"]
def predict_dataset(program, dataset):
    rows = []
    for i, ex in tqdm(enumerate(dataset), total=len(dataset)):
        out = program(sentence=ex.sentence)
        label = out.polarization
        rows.append({
            "id": test_df.iloc[i]["id"],
            "polarization": label2id(label),
        })
    return pd.DataFrame(rows)



# after you choose which model to use (classify, optimized_classify, etc.)
test_preds_df = predict_dataset(classify, raw_test)

print(test_preds_df.head())

  0%|          | 0/160 [00:00<?, ?it/s]

                                     id  polarization
0  eng_f66ca14d60851371f9720aaf4ccd9b58             0
1  eng_3a489aa7fed9726aa8d3d4fe74c57efb             0
2  eng_95770ff547ea5e48b0be00f385986483             0
3  eng_2048ae6f9aa261c48e6d777bcc5b38bf             0
4  eng_07781aa88e61e7c0a996abd1e5ea3a20             0


In [18]:
test_preds_df.to_csv(f"subtask1_{lang}_dspy_{student_lm_name.split("/")[-1]}.csv", index=False)

# Using MIPROv2

In [19]:
from dspy.teleprompt import MIPROv2

mipro = MIPROv2(
    metric=accuracy_metric,
    auto="light",
    teacher_settings=dict(lm=teacher_lm),
    prompt_model=student_lm
)

optimized_prog = mipro.compile(
    student=dspy.Predict(Polarization),
    trainset=raw_train,
    valset=raw_val,
    requires_permission_to_run=False,
)

# # After optimization, compute your full metrics dict
# metrics_val_opt = eval_metrics_on_dataset(optimized_prog, raw_val)

# print("Optimized validation metrics:", metrics_val_opt)

2025/12/05 19:06:58 WARNING dspy.teleprompt.mipro_optimizer_v2: 'requires_permission_to_run' is deprecated and will be removed in a future version.
2025/12/05 19:06:58 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 10
minibatch: True
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 100

2025/12/05 19:06:58 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/12/05 19:06:58 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/12/05 19:06:58 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...


Bootstrapping set 1/6
Bootstrapping set 2/6
Bootstrapping set 3/6


  1%|          | 5/515 [00:07<13:17,  1.56s/it]


Bootstrapped 4 full traces after 5 examples for up to 1 rounds, amounting to 5 attempts.
Bootstrapping set 4/6


  1%|          | 3/515 [00:04<13:24,  1.57s/it]


...

2025/12/05 19:13:55 INFO dspy.evaluate.evaluate: Average Metric: 29 / 35 (82.9%)
2025/12/05 19:13:55 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 82.86 on minibatch of size 35 with parameters ['Predictor 0: Instruction 2', 'Predictor 0: Few-Shot Set 4'].
2025/12/05 19:13:55 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [85.71, 82.86, 82.86, 77.14, 80.0, 91.43, 80.0, 85.71, 82.86]
2025/12/05 19:13:55 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [82.0, 85.0]
2025/12/05 19:13:55 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 85.0
2025/12/05 19:13:55 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/12/05 19:13:55 INFO dspy.teleprompt.mipro_optimizer_v2: == Trial 12 / 13 - Minibatch ==



Average Metric: 29.00 / 35 (82.9%): 100%|██████████| 35/35 [00:00<00:00, 3995.44it/s]

2025/12/05 19:13:55 INFO dspy.evaluate.evaluate: Average Metric: 29 / 35 (82.9%)
2025/12/05 19:13:55 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 82.86 on minibatch of size 35 with parameters ['Predictor 0: Instruction 1', 'Predictor 0: Few-Shot Set 3'].
2025/12/05 19:13:55 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [85.71, 82.86, 82.86, 77.14, 80.0, 91.43, 80.0, 85.71, 82.86, 82.86]
2025/12/05 19:13:55 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [82.0, 85.0]
2025/12/05 19:13:55 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 85.0
2025/12/05 19:13:55 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/12/05 19:13:55 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 13 / 13 - Full Evaluation =====
2025/12/05 19:13:55 INFO dspy.teleprompt.mipro_optimizer_v2: Doing full eval on next top averaging program (Avg Score: 87.14500000000001) from minibatch trials...



Average Metric: 85.00 / 100 (85.0%): 100%|██████████| 100/100 [00:08<00:00, 12.00it/s]

2025/12/05 19:14:03 INFO dspy.evaluate.evaluate: Average Metric: 85 / 100 (85.0%)
2025/12/05 19:14:03 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [82.0, 85.0, 85.0]
2025/12/05 19:14:03 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 85.0
2025/12/05 19:14:03 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/12/05 19:14:03 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/12/05 19:14:03 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 85.0!


In [20]:
ids = test_df["id"]
def predict_dataset(program, dataset):
    rows = []
    for i, ex in tqdm(enumerate(dataset), total=len(dataset)):
        out = program(sentence=ex.sentence)
        label = out.polarization
        rows.append({
            "id": test_df.iloc[i]["id"],
            "polarization": label2id(label),
        })
    return pd.DataFrame(rows)


# after you choose which model to use (classify, optimized_classify, etc.)
test_preds_df = predict_dataset(optimized_prog, raw_test)

print(test_preds_df.head())
test_preds_df.to_csv(f"subtask1_{lang}_dspy_miprov2_student{student_lm_name.split('/')[-1]}_teacher{teacher_lm_name.split('/')[-1]}.csv", index=False)

  0%|          | 0/160 [00:00<?, ?it/s]

                                     id  polarization
0  eng_f66ca14d60851371f9720aaf4ccd9b58             0
1  eng_3a489aa7fed9726aa8d3d4fe74c57efb             0
2  eng_95770ff547ea5e48b0be00f385986483             0
3  eng_2048ae6f9aa261c48e6d777bcc5b38bf             0
4  eng_07781aa88e61e7c0a996abd1e5ea3a20             0


In [24]:
# Save optimize program for future use
optimized_prog.save(f"optimized_subtask1_{lang}_dspy_miprov2_student{student_lm_name.split('/')[-1]}_teacher{teacher_lm_name.split('/')[-1]}.json")

# load the optimized program
save_path = f"optimized_subtask1_{lang}_dspy_miprov2_student{student_lm_name.split('/')[-1]}_teacher{teacher_lm_name.split('/')[-1]}.json"
optimized_prog_loaded = dspy.Predict(Polarization)